In [ ]:
import tflscripts
from tflscripts import take_percentage_of_data, read_dataset, filter_by_features, X_sort, filter_by_activities_transfer, classify, easy_domain_adaptation_update_dataframes, take_multiple_percentages_of_data, concat_and_reindex, filter_by_activities, read_and_filter_dataset, build_pipeline
from sklearn.metrics import accuracy_score
import numpy as np
from scipy.stats import mode
import pandas as pd

In [468]:
use_features = 'microphone.*index_mass_quantile'
force_columns = None
use_columns = None
use_activities = [0,1,2,3,4,5,7]

training_source_data_ratio = 0.6
training_target_data_ratio = 0.6
testing_target_data_ratio = 0.4

tflscripts.set_dataset_folder('/home/giotto/transfer-learning-playground/datasets/')

In [493]:
sources = [
    {
        'dataset': 'synergy-final-iter1',
        'device': 'xdk_1'
    },
    {
        'dataset': 'synergy-final-iter1',
        'device': 'xdk_2'
    },
    {
        'dataset': 'synergy-final-iter1',
        'device': 'xdk_3'
    },
    {
        'dataset': 'synergy-final-iter2',
        'device': 'xdk_1'
    },
    {
        'dataset': 'synergy-final-iter2',
        'device': 'xdk_2'
    },
    {
        'dataset': 'synergy-final-iter2',
        'device': 'xdk_3'
    },
    {
        'dataset': 'synergy-final-iter3',
        'device': 'xdk_1'
    },
    {
        'dataset': 'synergy-final-iter3',
        'device': 'xdk_2'
    },
    {
        'dataset': 'synergy-final-iter3',
        'device': 'xdk_3'
    },
    {
        'dataset': 'robotics-final',
        'device': 'xdk_1'
    },
    {
        'dataset': 'robotics-final',
        'device': 'xdk_2'
    },
    {
        'dataset': 'robotics-final',
        'device': 'xdk_3'
    },
    {
        'dataset': 'robotics-final',
        'device': '128.237.246.127'
    },
    {
        'dataset': 'robotics-final',
        'device': '128.237.247.134'
    }
]

target_dataset = 'scott-final-iter1'
target_device = '128.237.247.134'
target_teacher = 'xdk_2'

In [494]:
source_ppls = []

for source in sources:
    # read datasets
    df, df_labels = read_and_filter_dataset(
        source['dataset'],
        source['device'],
        use_features=use_features,
        force_columns=force_columns,
        use_columns=use_columns,
        use_activities=use_activities,
        scale=True,
        with_feature_selection=False)

    ppl = build_pipeline(df, df_labels, scale=True)

    source_ppls.append(ppl)

In [495]:
df_target, df_target_labels = read_and_filter_dataset(
        target_dataset,
        target_device,
        use_features=use_features,
        force_columns=force_columns,
        use_columns=use_columns,
        use_activities=use_activities,
        scale=True,
        with_feature_selection=False)

In [496]:
all_probs = [ppl.predict_proba(df_target) for ppl in source_ppls]
for i, probs in enumerate(all_probs):
    df_target['source_' + str(i) + '_pred'] = \
        [ppl.classes_[np.argmax(prob)] for prob in probs]
    df_target['source_' + str(i) + '_prob'] = \
        [np.max(prob) for prob in probs]

df_target = df_target.filter(regex='source_')

In [529]:
dfs = take_multiple_percentages_of_data(df_target, df_target_labels, [0.7, 0.3])
df_train, df_train_labels = dfs[0]
df_test, df_test_labels = dfs[1]

ppl = build_pipeline(df_train, df_train_labels, scale=True)
accuracy_score(df_test_labels['label'], ppl.predict(df_test))

0.85611510791366907

In [498]:
preds = [mode(preds)[0][0] for preds in df_target.filter(regex='source_.*_pred').values]

accuracy_score(df_target_labels['label'], preds)

0.69396551724137934

In [499]:
args = [np.argmax(probs) for probs in df_target.filter(regex='source_.*_prob').values]
preds = [preds[args[i]] for i, preds in enumerate(df_target.filter(regex='source_.*_pred').values)]
accuracy_score(df_target_labels['label'], preds)

0.70905172413793105

In [500]:
preds = df_target.filter(regex='source_.*_pred').values
pot = [label in preds[i] for i, label in enumerate(df_target_labels['label'])]
len([p for p in pot if p]) / len(pot)

0.9008620689655172

In [ ]:
df_target, df_target_labels = read_and_filter_dataset(
        target_dataset,
        target_device,
        use_features=use_features,
        force_columns=force_columns,
        use_columns=use_columns,
        use_activities=use_activities,
        scale=True,
        with_feature_selection=False)
df_target_labels